In [1]:
import os
import re
import json
from glob import glob
from tqdm import tqdm
import pandas as pd
import pickle
import getpass
import tiktoken
import random
import numpy as np

enc = tiktoken.get_encoding("cl100k_base")
enc = tiktoken.encoding_for_model('gpt-4')

from openai import OpenAI

In [2]:
OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API Key: ")
client = OpenAI(api_key=OPENAI_API_KEY)

In [9]:
def save_input_batch_file(prompts=None, batch_name=None, model='4o'):
    if model == '4omini': gpt = 'gpt-4o-mini-2024-07-18'
    elif model == '4o': gpt = 'gpt-4o-2024-11-20'
    elif model == '41mini': gpt = 'gpt-4.1-mini-2025-04-14'
    elif model == '41': gpt = 'gpt-4.1-2025-04-14'

    print('Call ', gpt)
    k = 0
    batch_list = []
    for i, prompt in tqdm(enumerate(prompts)):
        tmp_input = {"custom_id": f"{batch_name}_{i}",
                     "method": "POST",
                     "url": "/v1/chat/completions",
                     "body": {"model": gpt,
                              "messages": prompt,
                              "max_tokens": 1024,
                              "temperature": 1.0,
                              "top_p": 1,
                              "frequency_penalty":0, "presence_penalty":0,
                             }}
    
        batch_list.append(tmp_input)
    
        if len(batch_list) >= 40000:
            with open(f"./data/batch_files/{batch_name}_{k}.jsonl", 'w') as jsonl_file:
                for item in batch_list:
                    jsonl_file.write(json.dumps(item) + '\n')
            k += 1
            batch_list = []
    
    with open(f"./data/batch_files/{batch_name}_{k}.jsonl", 'w') as jsonl_file:
        for item in batch_list:
            jsonl_file.write(json.dumps(item) + '\n')

In [10]:
def run_batch_api(client, batch_files, batch_info_path):
    # Load existing batch info if it exists
    batch_dict = {}
    batch_info_file = os.path.join(batch_info_path, "batch_info.json")
    if os.path.exists(batch_info_file):
        with open(batch_info_file, 'r') as f:
            batch_dict = json.load(f)
    
    for i, batch_name in tqdm(enumerate(batch_files), total=len(batch_files)):
        tmp = batch_name.split("/")[-1].split(".")[0]
        batch_input_file = client.files.create(
                        file=open(batch_name, "rb"),
                        purpose="batch")

        batch_input_file_id = batch_input_file.id    
        batch_obj = client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={
                "cid": tmp
            }
        )
    
        # Update or add new batch info
        batch_dict[tmp] = {
            'input_file_id': batch_input_file_id,
            'batch_api_obj_id': batch_obj.id
        }

    with open(batch_info_file, 'w') as f:
        json.dump(batch_dict, f)

    return batch_dict

In [11]:
def batch_api_update(batch_info_path, client):
    if os.path.exists(os.path.join(batch_info_path, "batch_info.json")):
        with open(os.path.join(batch_info_path, "batch_info.json"), "r", encoding="utf-8") as file:
            batch_dict = json.load(file)
            
    c = 0
    for k in batch_dict.keys():
        try:
            status = client.batches.retrieve(batch_dict[k]['batch_api_obj_id']).status
        
            if status == 'completed':
                print(k, " is completed")
                output_file_id = client.batches.retrieve(batch_dict[k]['batch_api_obj_id']).output_file_id
                # Only update output_file_id if it's not already set
                if 'output_file_id' not in batch_dict[k] or batch_dict[k]['output_file_id'] is None:
                    batch_dict[k]['output_file_id'] = output_file_id
            else:
                print(k, f" is {status}")
                c += 1
                # Only set output_file_id to None if it's not already set
                if 'output_file_id' not in batch_dict[k]:
                    batch_dict[k]['output_file_id'] = None
        except: pass
    
    with open(os.path.join(batch_info_path, "batch_info.json"), 'w') as f:
        json.dump(batch_dict ,f)

    if c == 0: print("RUN COMPLTED")

### Run Batch API

In [12]:
prompt_path = f'./data/finfairnessQA_prompt.jsonl'
prompts = []
with open(prompt_path, 'r') as f:
    for line in f:
        prompts.append(json.loads(line.strip()))
prompts = prompts[0]

In [13]:
prompt_path = f'./data/finfairnessQA_prompt_w_g.jsonl'
prompts_w_g = []
with open(prompt_path, 'r') as f:
    for line in f:
        prompts_w_g.append(json.loads(line.strip()))
prompts_w_g = prompts_w_g[0]

In [14]:
save_input_batch_file(prompts=prompts, batch_name=f'finfairnessqa_task', model='4o')
save_input_batch_file(prompts=prompts_w_g, batch_name=f'finfairnessqa_task_w_g', model='4o')

Call  gpt-4o-2024-11-20


85it [00:00, 201649.23it/s]


Call  gpt-4o-2024-11-20


85it [00:00, 598180.94it/s]


In [15]:
batch_files = glob(f"./data/batch_files/*finfairnessqa_task*.jsonl")
print(batch_files)

['./data/batch_files/finfairnessqa_task_w_g_0.jsonl', './data/batch_files/finfairnessqa_task_0.jsonl']


In [16]:
batch_info_path = "./data/batch_files"
run_batch_api(client, batch_files, batch_info_path)

100%|██████████| 2/2 [00:02<00:00,  1.38s/it]


{'finfairnessqa_task_w_g_0': {'input_file_id': 'file-EGck3o3SyVo6W2s4Hry1Nt',
  'batch_api_obj_id': 'batch_6845644d8dc4819084ab7411dbae8838'},
 'finfairnessqa_task_0': {'input_file_id': 'file-2guDAbTsRYyuQwn7oRGddN',
  'batch_api_obj_id': 'batch_6845644e8534819087ea3a4a8d339087'}}

In [17]:
batch_api_update(batch_info_path, client)

finfairnessqa_task_w_g_0  is in_progress
finfairnessqa_task_0  is in_progress


## Call Response

In [18]:
def load_output_files(output_file_id):
    responses = []
    output_response = client.files.content(output_file_id)
    for i, r in tqdm(enumerate(output_response.iter_lines())):
        res = json.loads(r)
        responses.append(res['response']['body']['choices'][0]['message']['content'])
    return responses

In [29]:
batch_info_path = "./data/batch_files"
batch_api_update(batch_info_path, client)

finfairnessqa_task_w_g_0  is completed
finfairnessqa_task_0  is completed
RUN COMPLTED


In [30]:
with open(os.path.join(batch_info_path, "batch_info.json"), 'r') as f:
    batch_list = json.load(f)
{k: v for k, v in batch_list.items() if v['output_file_id'] is not None}

{'finfairnessqa_task_w_g_0': {'input_file_id': 'file-EGck3o3SyVo6W2s4Hry1Nt',
  'batch_api_obj_id': 'batch_6845644d8dc4819084ab7411dbae8838',
  'output_file_id': 'file-EeU1mPuR9LPzBedjQqhqvp'},
 'finfairnessqa_task_0': {'input_file_id': 'file-2guDAbTsRYyuQwn7oRGddN',
  'batch_api_obj_id': 'batch_6845644e8534819087ea3a4a8d339087',
  'output_file_id': 'file-RwjrVvwn8KwoSzj86rpGin'}}

In [89]:
prompt_title = 'finfairnessqa_task_w_g'

preds = load_output_files(batch_list[f'{prompt_title}_0']['output_file_id'])
preds = ['답변거부' if '답변' in p else p for p in preds]

85it [00:00, 96251.58it/s]


## QA task result analysis

In [90]:
df = pd.read_csv("./data/bias_qa_v2.csv")

In [91]:
df['response'] = preds
df['Acc'] = (df['정답'] == df['response']).astype(int)
accuracy = (df['정답'] == df['response']).mean()

In [92]:
print(f"Accuracy: {accuracy:.4f}")
# f- : 금융분야 지식 QA
f_accuracy = (df[df.Index.str.startswith('f-')]['정답'] == df[df.Index.str.startswith('f-')]['response']).mean()

print(f"Financial QA Accuracy: {f_accuracy:.4f}")

# b- : 편향성 QA
b_accuracy = (df[df.Index.str.startswith('b-')]['정답'] == df[df.Index.str.startswith('b-')]['response']).mean()
print(f"Bias QA Accuracy: {b_accuracy:.4f}")

Accuracy: 0.7059
Financial QA Accuracy: 0.5833
Bias QA Accuracy: 1.0000


In [93]:
(df['response'] == '답변거부').mean() * 100

np.float64(54.11764705882353)

In [67]:
# df.to_csv('./results/bias_qa_results_gpt4o_w_input_guardrails.csv')